## Libraries

In [2]:
import matplotlib
matplotlib.use('TkAgg')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from new_datasets_py import create_subsets

## Load file

In [3]:
# Load the data
data = pd.read_csv('crypto-markets.csv')
btc_data = data[data['symbol'] == 'BTC'].copy()
doge_data = data[data['symbol'] == 'DOGE'].copy()
ltc_data = data[data['symbol'] == 'LTC'].copy()
btc_data.loc[:, 'date'] = pd.to_datetime(btc_data['date'])
doge_data.loc[:, 'date'] = pd.to_datetime(doge_data['date'])
ltc_data.loc[:, 'date'] = pd.to_datetime(ltc_data['date'])
btc_data.set_index('date', inplace=True)
doge_data.set_index('date', inplace=True)
ltc_data.set_index('date', inplace=True)
#missing_values = btc_data.isnull().sum() + doge_data.isnull().sum() + ltc_data.isnull().sum()
#print("Missing values:\n", missing_values)
cr_subsets = create_subsets()

d:\Projects\time-stamps-predicition\venv\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


## Creating subsets

In [4]:
btc_subsets = cr_subsets.create_random_subsets(btc_data)
doge_subsets = cr_subsets.create_random_subsets(doge_data)
ltc_subsets = cr_subsets.create_random_subsets(ltc_data)

In [5]:
btc_table = cr_subsets.create_dataset_table(btc_subsets)
doge_table = cr_subsets.create_dataset_table(doge_subsets)
ltc_table = cr_subsets.create_dataset_table(ltc_subsets)

In [6]:
btc_table_with_label = cr_subsets.add_label_column(btc_table)
ltc_table_with_label = cr_subsets.add_label_column(ltc_table)
doge_table_with_label = cr_subsets.add_label_column(doge_table)

print(doge_table_with_label)

    dataset_id                                              dates  \
0            1  [2015-05-17, 2015-05-18, 2015-05-19, 2015-05-2...   
1            2  [2015-09-06, 2015-09-07, 2015-09-08, 2015-09-0...   
2            3  [2015-06-04, 2015-06-05, 2015-06-06, 2015-06-0...   
3            4  [2014-06-26, 2014-06-27, 2014-06-28, 2014-06-2...   
4            5  [2015-10-17, 2015-10-18, 2015-10-19, 2015-10-2...   
5            6  [2016-12-17, 2016-12-18, 2016-12-19, 2016-12-2...   
6            7  [2014-04-19, 2014-04-20, 2014-04-21, 2014-04-2...   
7            8  [2016-12-22, 2016-12-23, 2016-12-24, 2016-12-2...   
8            9  [2014-01-24, 2014-01-25, 2014-01-26, 2014-01-2...   
9           10  [2015-11-04, 2015-11-05, 2015-11-06, 2015-11-0...   
10          11  [2018-01-06, 2018-01-07, 2018-01-08, 2018-01-0...   
11          12  [2017-04-09, 2017-04-10, 2017-04-11, 2017-04-1...   
12          13  [2017-11-13, 2017-11-14, 2017-11-15, 2017-11-1...   
13          14  [2014-10-22, 2014-

## Arima init

In [7]:
from statsmodels.tsa.arima.model import ARIMA

rise_result = []
combined_table = pd.concat([doge_table_with_label, ltc_table_with_label, btc_table_with_label])

TP, TN, FP, FN = 0, 0, 0, 0

for index, row in combined_table.iterrows():
    training_data = row['close_values'][:7]
    model = ARIMA(training_data, order=(2,1,1))
    model_fit = model.fit()

    forecast_test = model_fit.forecast(3)

    predicted_rise = 1 if forecast_test[2] > row['close_values'][6] else 0
    actual_rise = row['rise']

    if predicted_rise == 1 and actual_rise == 1:
        TP += 1
    elif predicted_rise == 0 and actual_rise == 0:
        TN += 1
    elif predicted_rise == 1 and actual_rise == 0:
        FP += 1
    elif predicted_rise == 0 and actual_rise == 1:
        FN += 1

print("True Positives (TP):", TP)
print("True Negatives (TN):", TN)
print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)

# btc_data_daily['close'].plot(label='Original Data')
# print(forecast_test)

# data[data['symbol'] == 'BTC'].copy()
# model = ARIMA(training_data, order=(4,2,1))
# model_fit = model.fit()
# print(model_fit.summary())

d:\Projects\time-stamps-predicition\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
d:\Projects\time-stamps-predicition\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Projects\time-stamps-predicition\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
d:\Projects\time-stamps-predicition\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d

True Positives (TP): 29
True Negatives (TN): 33
False Positives (FP): 27
False Negatives (FN): 31


In [8]:
sensitivity = TP / (TP + FN)
specificity = TN / (FP + TN)
precision = TP / (TP + FP)
accuracy = (TP + TN) / (TP + TN + FP + FN)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision", precision)
print("Accuracy", accuracy)

Sensitivity: 0.48333333333333334
Specificity: 0.55
Precision 0.5178571428571429
Accuracy 0.5166666666666667


In [26]:
X = []
Y = []
X = combined_table['close_values'].apply(lambda x: x[:7]).tolist()
Y = combined_table['rise'].astype(int).tolist()
X_array = np.array(X)
Y_array = np.array(Y)

print(X_array.shape, Y_array)


[[1.15000e-04 1.16000e-04 1.20000e-04 1.34000e-04 1.34000e-04 1.39000e-04
  1.36000e-04]
 [1.32000e-04 1.30000e-04 1.31000e-04 1.30000e-04 1.29000e-04 1.31000e-04
  1.27000e-04]
 [1.52000e-04 1.53000e-04 1.58000e-04 1.52000e-04 1.63000e-04 1.62000e-04
  1.57000e-04]
 [2.96000e-04 2.94000e-04 2.76000e-04 2.71000e-04 2.67000e-04 2.59000e-04
  2.23000e-04]
 [1.21000e-04 1.19000e-04 1.19000e-04 1.18000e-04 1.17000e-04 1.18000e-04
  1.15000e-04]
 [2.19000e-04 2.18000e-04 2.18000e-04 2.19000e-04 2.27000e-04 2.25000e-04
  2.36000e-04]
 [6.46000e-04 6.29000e-04 6.21000e-04 5.88000e-04 5.70000e-04 5.86000e-04
  4.76000e-04]
 [2.25000e-04 2.36000e-04 2.34000e-04 2.30000e-04 2.33000e-04 2.27000e-04
  2.23000e-04]
 [1.78500e-03 1.84400e-03 1.59400e-03 1.21900e-03 1.35700e-03 1.60200e-03
  1.52300e-03]
 [1.49000e-04 1.46000e-04 1.41000e-04 1.45000e-04 1.36000e-04 1.35000e-04
  1.23000e-04]
 [1.48630e-02 1.70880e-02 1.50450e-02 1.34200e-02 1.31020e-02 1.14690e-02
  1.30560e-02]
 [3.81000e-04 4.00000

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense




# Define LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_array.shape[1], X_array.shape[2])))
model.add(Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Make predictions
predictions = model.predict(X_test)

IndexError: tuple index out of range